In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import math
import numpy as np
from matplotlib.pyplot import figure
from scipy.interpolate import interp1d
from scipy import stats
from numpy import random
import sys
import os

In [ ]:
# Constant values
pwd_path = os.getcwd()
output_path = pwd_path + '/../data/weighted-datasets'
output_path = pwd_path + '/../data/non-weighted-datasets'

In [ ]:
def dataset_name_to_index(dataset_name):
    return dataset_name

In [ ]:
# A note regarding stats_norm_interval_division_factor_const:
# The stats_norm_interval_division_factor_const parameter is set to select the MPIW width / PICP
# working point abd is set differently in different datasets - This is a hyper-parameter of the model selected
# by the user.

#dataset_name = 'A'
#stats_norm_interval_division_factor_const = 5

#dataset_name = 'B'
#stats_norm_interval_division_factor_const = 10

#dataset_name = 'C'
#stats_norm_interval_division_factor_const = 10

dataset_name = 'D'
stats_norm_interval_division_factor_const = 10

#dataset_name = 'E'
#stats_norm_interval_division_factor_const = 5

In [ ]:
csv_name = output_path + '/snr_predictions_vs_losses_' + dataset_name + '.csv'
df = pd.read_csv(csv_name) 

# Save all output vectors to this CSV
csv_vectors_output_filename = output_path + '/snr_predictions_results_dataset_' +\
                              dataset_name + '.csv'
df_snr_results = pd.DataFrame()

csv_name_validation = output_path + '/snr_predictions_vs_losses_validation_' + dataset_name + '.csv'
df_validation = pd.read_csv(csv_name_validation)

infinity_loss_definition = 10e16

# Fix some bug (these are the validation losses, irrelevant for what we are trying to achieve)
if ('losses_val' not in df.columns):
    df['losses_val'] = 1

df = df[df['losses_val'] < infinity_loss_definition]
df = df[df['losses_val'] > -infinity_loss_definition]

fig, ax = plt.subplots()
#ax.plot(SNR_predictions, losses,label="Prediction losses [%] vs. SNR per prediction")

ax.scatter(df['SNR_predictions'], df['losses_val'])
#ax.title('Prediction losses [%] vs. SNR per prediction')
plt.gca().update(dict(title='Prediction losses [%] vs. SNR per prediction [Low SNR]'))
ax.legend(fontsize=8)
ax.set_xlabel("SNR")
ax.set_ylabel("loss %")

print('Variance of losses =', df['losses_val'].var(), df['losses_val'].mean())

In [ ]:
capacity1 = 0.5 * (math.log2(1 + 80))
capacity2 = 0.5 * (math.log2(1 + 4000))
capacity3 = 0.5 * (math.log2(1 + 10000))

print(capacity1, capacity2, capacity3)

In [ ]:
df['predictions_sample_test'].hist()

In [ ]:
#range_pred = df['predictions_sample'].max() - df['predictions_sample'].min() 

range_pred = 1000000 / 10000
print("error_pctg = ", 1/range_pred)
#print(range_pred / np.power(2,6))
#print(range_pred / np.power(2,3))
print("range_pred={}, log2(range_pred)={}".format(range_pred, math.log2(range_pred)))

In [ ]:
# Filter infinity values
print(df.shape)

df = df[df['SNR_predictions'] < 10000]

print(df.shape)
df.head()

In [ ]:
df_validation

In [ ]:
prediction_errors_val = (df_validation['y_val_actual'] - df_validation['y_val_pred'])
#prediction_errors_val = prediction_errors_val[np.abs(prediction_errors_val) / df_validation['y_val_actual'] < 0.5]
plt.hist(prediction_errors_val, bins = 200)
plt.xlabel('Prediction error')
plt.ylabel('Frequency')
output_file = output_path + '/prediction_error_hist_validation_' + dataset_name + '.png'
print(output_file)
plt.savefig(output_file)


dataset_prediction_error_level = np.var(prediction_errors_val)
print("prediction_errors_val = ", dataset_prediction_error_level)

In [ ]:
prediction_errors = (df['actual_sample_test'] - df['predictions_sample_test'])
prediction_errors = prediction_errors[np.abs(prediction_errors)/df['actual_sample_test'] < 0.5]
plt.hist(prediction_errors, bins = 200)
plt.xlabel('Prediction error')
plt.ylabel('Frequency')
output_file = output_path + '/prediction_error_hist_' + dataset_name + '.png'
print(output_file)
plt.savefig(output_file)

In [ ]:
#df['SNR'] = df['target_energy_level'] / df['noise_level']

# Calculate the SNR
mean_noise = df[df['noise_level'] > 1.000].mean()
df[df['noise_level'] == 1.000]['noise_level'] = mean_noise


# local samples noise level SNR
df['SNR'] = df['predictions_sample_test']**2 / df['noise_level']

# Global dataset noise level SNR
#df['SNR'] = df['predictions_sample_test']**2 / dataset_prediction_error_level 

SNR_hist = df['SNR'][df['SNR'] < 10000]
SNR_hist.hist(bins=40)

In [ ]:
df['num_queries_in_subspace'].hist(bins=100)

In [ ]:
fig, ax = plt.subplots()

#df_filtered = df[df['SNR_predictions'] < 0.2*1e11]
#df_filtered = df_filtered[df_filtered['SNR_predictions'] > 0.4*1e10]
df_filtered = df[df['SNR'] < 100]
#df_filtered = df_filtered[df_filtered['SNR_predictions'] > 0.4*1e10]

ax.scatter(df_filtered['SNR'], df_filtered['losses_val'])
#ax.title('Prediction losses [%] vs. SNR per prediction')
plt.gca().update(dict(title='Prediction losses [%] vs. SNR per prediction [Low SNR]'))
ax.legend(fontsize=8)
ax.set_xlabel("SNR")
ax.set_ylabel("loss %")

print('Variance of losses =', df_filtered['losses_val'].var())

In [ ]:
fig, ax = plt.subplots()

#df_filtered = df[df['SNR_predictions'] > 1e13]
#df_filtered = df_filtered[df_filtered['SNR_predictions'] < 2e13]
df_filtered = df[df['SNR'] > 1000]
df_filtered = df_filtered[df_filtered['SNR'] < 10000]


ax.scatter(df_filtered['SNR'], df_filtered['losses_val'])
#ax.title('Prediction losses [%] vs. SNR per prediction')
plt.gca().update(dict(title='Prediction losses [%] vs. SNR per prediction [Medium SNR]', xlabel='x', ylabel='y'))
ax.legend(fontsize=8)
ax.set_xlabel("SNR")
ax.set_ylabel("loss %")
print('Variance of losses =', df_filtered['losses_val'].var())

In [ ]:
fig, ax = plt.subplots()

#df_filtered = df[df['SNR_predictions'] > 2e13]
#df_filtered = df_filtered[df_filtered['SNR_predictions'] < 4e13]
df_filtered = df[df['SNR'] > 10000]
df_filtered = df_filtered[df_filtered['SNR'] < 1000000]

ax.scatter(df_filtered['SNR'], df_filtered['losses_val'])
#ax.title('Prediction losses [%] vs. SNR per prediction')
plt.gca().update(dict(title='Prediction losses [%] vs. SNR per prediction [High SNR]', xlabel='x', ylabel='y'))
ax.legend(fontsize=8)
ax.set_xlabel("SNR")
ax.set_ylabel("loss %")
print('Variance of losses =', df_filtered['losses_val'].var())

In [ ]:
fig, ax = plt.subplots()

df_filtered = df[df['SNR'] > 1e6]
df_filtered = df_filtered[df_filtered['SNR'] < 1e13]

ax.scatter(df_filtered['SNR'], df_filtered['losses_val'])
#ax.title('Prediction losses [%] vs. SNR per prediction')
plt.gca().update(dict(title='Prediction losses [%] vs. SNR per prediction', xlabel='x', ylabel='y'))
ax.legend(fontsize=8)
ax.set_xlabel("SNR")
ax.set_ylabel("loss %")
print('Variance of losses =', df_filtered['losses_val'].var())

In [ ]:
(df['SNR'][df['SNR'] == 1].count())

In [ ]:
(df['SNR'][df['SNR'] != 1].count())

In [ ]:
fig, ax = plt.subplots()

#capacity = []
#capacity3 = 0.5 * (math.log2(1 + 10000))
# [-A......|......+A] == 1 bit
# 
# A + A*j  ==> 2 bits
# -A + A*j
# A - A*j
# -A - A*j

# X_real_target ==> Hunch(X_real_target) ==> Hunch(X_features) = X_pred_target = X_real_target + n 
# ==> such that, n is AWGN(0, variance)
# 
# Tx symbols bits <= shannon_capacity = 1.5 bits / channel usage 
# 
# global query space ==> target_real is always in range = [target_real_min, target_real_max]
# bits_in_range = log2((target_real_max - target_real_min) / 2^10)
#               = [0, 1, 2, .., K-1] ==> [0, 2^10, 2*2^10, ..., (K-1)*2^10]
# 
df['shannon_capacity'] = 0.5 * (np.log2(1 +  df['SNR']))
df['shannon_capacity'].hist(bins=20)

#ax.scatter(df_filtered['SNR'], df_filtered['losses'])
##ax.title('Prediction losses [%] vs. SNR per prediction')
#plt.gca().update(dict(title='Prediction losses [%] vs. Capacity (high bound of rate)', xlabel='x', ylabel='y'))
#ax.legend(fontsize=8)
#ax.set_xlabel("SNR")
#ax.set_ylabel("loss %")
#print('Variance of losses =', df_filtered['losses'].var())

In [ ]:
#df['actual_pred'] =  df['predictions_sample'] / (df['losses']/100)
#df['actual_pred'].hist(bins=10)

#df[df['shannon_capacity'] < 2]['losses'].mean()
#df[df['shannon_capacity'] > 2]['losses'].mean()

min_val = df_filtered['predictions_sample_test'].min()
max_val = df_filtered['predictions_sample_test'].max()
bits = np.log2(max_val - min_val)

loss_variance_as_function_of_capacity = []

# Build capacities list we want to run over
capacities = []
step_size = 2
initial_value = 0 * step_size
last_capacity = 7
#last_capacity = 20
for l in range(initial_value, last_capacity * step_size):
    capacities.append(l / step_size)
    
count_samples = []
loss_means = []
max_errors = []
prediction_losses_test = []
prediction_losses_mean_test = []
pctg_samples_larger_capacity_test = []
mean_num_queries_in_subspace_on_threshold_capacity = []
loss_variance_testset_on_threshold_capacity = []

mean_num_bits_required_testset_on_threshold_capacity = []

confidence_interval_per_capacity_low_bound = []
confidence_interval_per_capacity_high_bound = []
confidence_interval_per_capacity_length = []
confidence_interval_per_capacity_coverage_ratio = []

queries_per_capacity_total = []

mean_target_actual_value_on_capacity = []
median_target_actual_value_on_capacity = []
percentile_0_25_target_actual_value_on_capacity = []
percentile_0_50_target_actual_value_on_capacity = []
percentile_0_75_target_actual_value_on_capacity = []

# Calculate Hunch PI
Hunch_PI = np.sqrt(sum((df['actual_sample_test'] - df['predictions_sample_test'])**2) / len(df['predictions_sample_test']))
print("Hunch_PI = ", Hunch_PI)

total_hunch_pi_coverage_queries = 0
total_queries = 0

N = df['shannon_capacity'].count()
classification_df = pd.DataFrame()
for capacity in capacities:
    cond1 = (df['shannon_capacity'] >= capacity)
    cond2 = (df['shannon_capacity'] < (capacity + (1 / step_size)))
    df_filtered = df[cond1 & cond2]
    print(capacity)
    df_filtered['shannon_capacity'].hist()
    
    #df_filtered = df[(df['shannon_capacity'] >= capacity) & (df['shannon_capacity'] < (capacity+1))]
    df_filtered_all_larger = df_filtered
    #df_filtered_all_larger = df[(df['shannon_capacity'] >= capacity)]

    # Important: Prevent nans !!!!
    df_filtered = df_filtered[df_filtered['losses_val'] < infinity_loss_definition]
    df_filtered = df_filtered[df_filtered['losses_val'] > -infinity_loss_definition]

    df_filtered['minimum_capacity'] = capacity
    classification_df = pd.concat([classification_df, df_filtered], axis=0)

    loss_variance = np.var(df_filtered['losses_val'])
    loss_mean = np.mean(df_filtered['losses_val'])

    K = df_filtered_all_larger['shannon_capacity'].count()
    
    # Do we still have enough samples for statistics?
    if (K/N) < 0.01:
        continue

    pctg_samples_larger_capacity_test.append(K/N)

    loss_variance_as_function_of_capacity.append(loss_variance)
    count_samples.append(df_filtered['shannon_capacity'].count())
    loss_means.append(loss_mean)

    a = df_filtered_all_larger['actual_sample_test']
    b = df_filtered_all_larger['predictions_sample_test']

    losses_on_capacity = 100 * ((a - b) / a)
    
    mean_target_actual_value_on_capacity.append(df_filtered_all_larger['actual_sample_test'].mean())
    
    # TODO: Add Median, 25%, 50% and 75%
    median_target_actual_value_on_capacity.append(df_filtered_all_larger['actual_sample_test'].median())

    percentile_0_25_target_actual_value_on_capacity.append(np.percentile(df_filtered_all_larger['actual_sample_test'], 25))
    percentile_0_50_target_actual_value_on_capacity.append(np.percentile(df_filtered_all_larger['actual_sample_test'], 50))
    percentile_0_75_target_actual_value_on_capacity.append(np.percentile(df_filtered_all_larger['actual_sample_test'], 75))
    
    # number of samples used in validation space
    mean_num_queries_in_subspace_on_threshold_capacity.append(df_filtered_all_larger['num_queries_in_subspace'].mean())

    # number of bits required to represent {max - min}
    max_in_queries = df_filtered_all_larger['max_value_in_query_subspace_val']
    min_in_queries = df_filtered_all_larger['min_value_in_query_subspace_val']
    mean_num_bits = np.log2(abs(max_in_queries - min_in_queries).mean())
    mean_num_bits_required_testset_on_threshold_capacity.append(mean_num_bits)
    
    # Calculate CI (Confidence Interval) based on T student distribution - For each Shannon Capacity:
    #mean, sigma = np.mean(losses_on_capacity), np.std(losses_on_capacity)
    predictions_with_capacity_larger = df_filtered_all_larger['predictions_sample_test'].copy()
    sigma = np.std(predictions_with_capacity_larger) / stats_norm_interval_division_factor_const
    mean = np.mean(predictions_with_capacity_larger)
    conf_int = stats.norm.interval(0.95, loc=mean, scale=sigma)
    #print("Confidence Interval: ", conf_int)
    low_bound = conf_int[0]
    high_bound = conf_int[1]
    confidence_interval_per_capacity_low_bound.append(low_bound)
    confidence_interval_per_capacity_high_bound.append(high_bound)
 
    # The total number of queries per capacity
    queries_per_capacity_total.append(len(predictions_with_capacity_larger))

    # Calculate the interval length mean average for each shannon capacity and plot in plot 6 
    # and put a number in each shannon capacity
    confidence_interval_per_capacity_length.append(high_bound - low_bound)

    # Coverage ==> Number of times where a target falls between the prediction interval for each 
    #              Shannon Capacity divided by the number of queries for each shannon capacity
    #
    coverage_samples = predictions_with_capacity_larger[predictions_with_capacity_larger < high_bound]
    coverage_samples = predictions_with_capacity_larger[predictions_with_capacity_larger >= low_bound]
    coverage = coverage_samples.count()
    confidence_interval_coverage_ratio = coverage / predictions_with_capacity_larger.count()
    confidence_interval_per_capacity_coverage_ratio.append(confidence_interval_coverage_ratio)
    
    # Add total samples that fall into the HunchPI
    total_hunch_pi_coverage_queries += coverage
    total_queries += predictions_with_capacity_larger.count()   

    predictions_with_capacity_larger = df_filtered_all_larger['predictions_sample_test'].copy()
    #coverage_samples = predictions_with_capacity_larger[predictions_with_capacity_larger < v2]
    #coverage_samples = predictions_with_capacity_larger[predictions_with_capacity_larger >= v1]
    #coverage = coverage_samples.count()
    #confidence_interval_coverage_ratio = coverage / predictions_with_capacity_larger.count()
   
    # important: prevent nans
    losses_on_capacity = losses_on_capacity[losses_on_capacity < infinity_loss_definition]
    losses_on_capacity = losses_on_capacity[losses_on_capacity > -infinity_loss_definition]
    #print("losses_on_capacity = ", losses_on_capacity)
    
    loss_test = losses_on_capacity.std()
    loss_test_mean = losses_on_capacity.mean()

    # New variance when cutting off the capacities lower (i.e. they are read directly from DB with regular query)
    # prevent divide by 0
    if (N == 0):
        N = 1
    
    loss_variance_testset_on_threshold_capacity.append(loss_test * ((K/N)**2))

    prediction_losses_test.append(loss_test)
    prediction_losses_mean_test.append(loss_test_mean)
    
    #min_val = df_filtered['predictions_sample_val'].min()
    #max_val = df_filtered['predictions_sample_val'].max()
    #bits = np.log2(max_val - min_val)
    max_error = (2**bits) / (2**capacity)
    max_errors.append(max_error)
    #print("capacity={}, min_val={}, max_val={}, bits={} max_error={}, loss_test={}, loss_test_mean={}".\
    #      format(capacity, min_val, max_val, bits, max_error,loss_test, loss_test_mean))

if 0:
    print("capacities = ", capacities)
    print("loss variance for each capacity in [capacities[i],capacities[i]+1] = ", loss_variance_as_function_of_capacity)
    print("loss mean for each capacity in [capacities[i],capacities[i]+1] = ", loss_means)

    print("prediction_losses_test = ", prediction_losses_test)
    print("count_samples = ", count_samples)

    print("max_errors > loss_variance_as_function_of_capacity ? ==> ",\
          max_errors > loss_variance_as_function_of_capacity)

    print("pctg_samples_larger_capacity_test = ", pctg_samples_larger_capacity_test)

fig, ax = plt.subplots(10)
#plt.rcParams["figure.figsize"] = (12,12)
plt.gcf().set_size_inches(20, 60, forward=True) 

# Cut capacities
capacities = capacities[0:len(prediction_losses_test)]

x_label_name_shannon_capacity = "Minimum Shannon's capacity"

plot_idx = 0

# plot 1: prediction_loss (test data)
#ax[plot_idx].scatter(capacities, loss_variance_as_function_of_capacity)
ax[plot_idx].plot(capacities, prediction_losses_test)
ax[plot_idx].legend(['prediction_loss (test data)'])
ax[plot_idx].legend(fontsize=8)
ax[plot_idx].set_xlabel(x_label_name_shannon_capacity)
ax[plot_idx].set_ylabel("Loss Variance")
ax[plot_idx].set_title(dataset_name)

plot_idx += 1

# plot 2:% samples with larger capacity
ax[plot_idx].plot(capacities, pctg_samples_larger_capacity_test)
ax[plot_idx].legend(fontsize=8)
ax[plot_idx].legend()
ax[plot_idx].legend(['% samples with larger capacity'])
ax[plot_idx].set_xlabel(x_label_name_shannon_capacity)
ax[plot_idx].set_ylabel("Percentage of samples in test")
ax[plot_idx].set_title(dataset_name)


plot_idx += 1

# plot 3: loss mean with larger capacity
ax[plot_idx].plot(capacities, prediction_losses_mean_test)
ax[plot_idx].legend(fontsize=8)
ax[plot_idx].legend()
ax[plot_idx].legend(['loss mean with larger capacity'])
ax[plot_idx].set_xlabel(x_label_name_shannon_capacity)
ax[plot_idx].set_ylabel("Prediction loss mean (test data)")
ax[plot_idx].set_title(dataset_name)

plot_idx += 1

# plot 4: loss mean with larger capacity (test data + DB query)
ax[plot_idx].plot(capacities, loss_variance_testset_on_threshold_capacity)
ax[plot_idx].legend(fontsize=8)
ax[plot_idx].legend()
ax[plot_idx].legend(['loss mean with larger capacity (test data + DB query)'])
ax[plot_idx].set_xlabel(x_label_name_shannon_capacity)
ax[plot_idx].set_ylabel("Prediction loss variance after applying (N/K)^2 (test data)")
ax[plot_idx].set_title(dataset_name)

plot_idx += 1

# plot 5: Mean of number of queries 
ax[plot_idx].plot(capacities, mean_num_queries_in_subspace_on_threshold_capacity)
ax[plot_idx].legend(fontsize=8)
ax[plot_idx].legend()
ax[plot_idx].legend(['Mean of number of queries in the same subspace in validation data'])
ax[plot_idx].set_xlabel(x_label_name_shannon_capacity)
ax[plot_idx].set_ylabel("Mean of num_queries_in_subspace in validation data")
ax[plot_idx].set_title(dataset_name)


plot_idx += 1

# plot 5: Mean of log2{max-min}
ax[plot_idx].plot(capacities, mean_num_bits_required_testset_on_threshold_capacity)
ax[plot_idx].legend(fontsize=8)
ax[plot_idx].legend()
ax[plot_idx].legend(['Mean of log2{max-min} in validation data with larger capacity'])
ax[plot_idx].set_xlabel(x_label_name_shannon_capacity)
ax[plot_idx].set_ylabel("Mean of of log2{max-min} in validation data")
ax[plot_idx].set_title(dataset_name)

plot_idx += 1

# plot 6: CI (Confidence Interval)
confidence_interval_per_capacity_length_text = 'CI low bound vs. high bound length, division_factor=' +\
                                               str(stats_norm_interval_division_factor_const)
#print(confidence_interval_per_capacity_low_bound)
ax[plot_idx].plot(capacities, confidence_interval_per_capacity_low_bound)
ax[plot_idx].plot(capacities, confidence_interval_per_capacity_high_bound)
ax[plot_idx].plot(capacities, mean_target_actual_value_on_capacity)
ax[plot_idx].plot(capacities, confidence_interval_per_capacity_length)

ax[plot_idx].plot(capacities, median_target_actual_value_on_capacity)
ax[plot_idx].plot(capacities, percentile_0_25_target_actual_value_on_capacity)
ax[plot_idx].plot(capacities, percentile_0_50_target_actual_value_on_capacity)
ax[plot_idx].plot(capacities, percentile_0_75_target_actual_value_on_capacity)

ax[plot_idx].legend(fontsize=8)
ax[plot_idx].legend()
ax[plot_idx].legend(['CI in validation data with larger capacity LOW bound', 'CI in validation data with larger capacity HIGH bound',
                     'Mean of actual target value >= capacity', confidence_interval_per_capacity_length_text,
                     'Median of actual target value >= capacity',
                     '0.25 percentile of actual target value >= capacity',
                     '0.50 percentile of actual target value >= capacity',
                     '0.75 percentile of actual target value >= capacity'])
ax[plot_idx].set_xlabel(x_label_name_shannon_capacity)
ax[plot_idx].set_ylabel("CI in validation data")
ax[plot_idx].set_title(dataset_name)


plot_idx += 1

# Calculate the interval length mean average for each shannon capacity and plot in plot 6 and put a number in each shannon capacity

# plot 7: CI (Confidence Interval coverage ratio)
#print(confidence_interval_per_capacity_low_bound)
ax[plot_idx].plot(capacities, confidence_interval_per_capacity_coverage_ratio)
ax[plot_idx].legend(fontsize=8)
ax[plot_idx].legend()
ax[plot_idx].legend(['confidence interval coverage ratio'])
ax[plot_idx].set_xlabel(x_label_name_shannon_capacity)
ax[plot_idx].set_ylabel("CI in validation data")
ax[plot_idx].set_title(dataset_name)

plot_idx += 1



In [ ]:
# Save to results CSV
df_snr_results['capacities'] = capacities
df_snr_results['prediction_losses_test'] = prediction_losses_test
df_snr_results['pctg_samples_larger_capacity_test'] = pctg_samples_larger_capacity_test
df_snr_results['prediction_losses_mean_test'] = prediction_losses_mean_test
df_snr_results['loss_variance_testset_on_threshold_capacity'] = loss_variance_testset_on_threshold_capacity
df_snr_results['mean_num_queries_in_subspace_on_threshold_capacity'] = mean_num_queries_in_subspace_on_threshold_capacity
df_snr_results['mean_num_bits_required_testset_on_threshold_capacity'] = mean_num_bits_required_testset_on_threshold_capacity
df_snr_results['confidence_interval_per_capacity_coverage_ratio'] = confidence_interval_per_capacity_coverage_ratio
df_snr_results['confidence_interval_per_capacity_low_bound'] = confidence_interval_per_capacity_low_bound
df_snr_results['confidence_interval_per_capacity_high_bound'] = confidence_interval_per_capacity_high_bound
df_snr_results['mean_target_actual_value_on_capacity'] = mean_target_actual_value_on_capacity
df_snr_results['confidence_interval_per_capacity_length'] = confidence_interval_per_capacity_length
df_snr_results['median_target_actual_value_on_capacity'] = median_target_actual_value_on_capacity
df_snr_results['percentile_0_25_target_actual_value_on_capacity'] = percentile_0_25_target_actual_value_on_capacity
df_snr_results['percentile_0_50_target_actual_value_on_capacity'] = percentile_0_50_target_actual_value_on_capacity
df_snr_results['percentile_0_75_target_actual_value_on_capacity'] = percentile_0_75_target_actual_value_on_capacity
df_snr_results['confidence_interval_per_capacity_coverage_ratio'] = confidence_interval_per_capacity_coverage_ratio
df_snr_results['queries_per_capacity_total'] = queries_per_capacity_total

df_snr_results.to_csv(csv_vectors_output_filename, index = False)



In [ ]:
import seaborn as sns 
if 0:
    columns_names_y = ['prediction_losses_test',
           'pctg_samples_larger_capacity_test', 'prediction_losses_mean_test',
           'loss_variance_testset_on_threshold_capacity',
           'mean_num_queries_in_subspace_on_threshold_capacity',
           'mean_num_bits_required_testset_on_threshold_capacity',
           'confidence_interval_per_capacity_coverage_ratio',
           'confidence_interval_per_capacity_low_bound',
           'confidence_interval_per_capacity_high_bound',
           'mean_target_actual_value_on_capacity',
           'confidence_interval_per_capacity_length',
           'median_target_actual_value_on_capacity',
           'percentile_0_25_target_actual_value_on_capacity',
           'percentile_0_50_target_actual_value_on_capacity',
           'percentile_0_75_target_actual_value_on_capacity']
else:
    #,
    columns_names_y = ['prediction_losses_mean_test']
                      #'pctg_samples_larger_capacity_test'] #['prediction_losses_test',       


    
# reshape the dataframe 
#dfm = df_snr_results.melt(id_vars=columns_names_y)

import seaborn as sns
import matplotlib.pyplot as plt
  
# loading dataset
#data = sns.load_dataset("tips")
  
# draw pointplot

plot_results = df_snr_results.copy()
fig, ax = plt.subplots(figsize=(16,6))

# Normalize
if 0:
    max_val = 0
    for col in columns_names_y:
        if (max_val < max(plot_results[col])):
            max_val = max(plot_results[col])
else:
    max_val = 1

for col in columns_names_y:
    plot_results[col] = plot_results[col] / max_val

df = plot_results.melt("capacities", columns_names_y)

ax = sns.pointplot(data=df, x='capacities', y='value', hue="variable", ci='sd', dodge=0.25)

#for col in columns_names_y:
#    plot_results[col] = plot_results[col] / max(plot_results[col])
#    sns.pointplot(x='capacities', y=col, data=plot_results, ci='sd', dodge=0.25)
#c=sns.pointplot(data = df, x='Date', y='TotalConfirmed', color="b", 
#                label='Total Confirmed')
#d=sns.pointplot(data = df, x='Date', y='TotalDeath', color="r", 
#                label='Total Death')
#r=sns.pointplot(data = df, x='Date', y='TotalRecovered', color="g", 
#                label='Total Recovered')
ax.set_title('Dataset_' + dataset_name_to_index(dataset_name), fontsize=22, y=1.015)
ax.set_xlabel("Minimum Shannon's Capacity", labelpad=16)
#ax.set_ylabel('# of people', labelpad=16)
#ax.set(yscale="log")
t=plt.xticks(rotation=45)

# show the plot
plt.show()

#p = sns.pointplot(data=dfm, x='capacities', hue='prediction_losses_test', y='prediction_losses_test',
#                  ci='sd', dodge=0.25) #hue='g', ) #, y='value'
#p.set_title('Error bars are standard deviation') 
#p.legend(title='g', bbox_to_anchor=(1.05, 1), loc='upper left')


In [ ]:
df.head(100)

In [ ]:
fig, ax = plt.subplots(1)
#plt.gcf().set_size_inches(20, 60, forward=True) 

# Cut capacities
capacities = capacities[0:len(prediction_losses_test)]

plot_idx = 0

# plot 1: prediction_loss (test data)
#ax[plot_idx].scatter(capacities, loss_variance_as_function_of_capacity)
prediction_losses_test_normalized = prediction_losses_test / np.max(prediction_losses_test)
ax.plot(capacities, prediction_losses_test)
ax.legend(['prediction_loss (test data)'])
ax.legend(fontsize=8)
ax.set_xlabel(x_label_name_shannon_capacity)
ax.set_ylabel("Loss Variance (Normalized)")
ax.set_title(dataset_name_to_index(dataset_name))

filename_out = output_path + "/prediction_loss_test_" +\
        dataset_name_to_index(dataset_name) + ".png"
plt.savefig(filename_out)


In [ ]:


fig, ax = plt.subplots(1)
#plt.gcf().set_size_inches(20, 60, forward=True) 
plot_idx = 0

# plot 2:% samples with larger capacity
ax.plot(capacities, pctg_samples_larger_capacity_test)
ax.legend(fontsize=8)
ax.legend()
ax.legend(['% samples with larger capacity'])
ax.set_xlabel(x_label_name_shannon_capacity)
ax.set_ylabel("Percentage of samples in test")
ax.set_title(dataset_name_to_index(dataset_name))

filename_out = output_path + "/prediction_pctg_samples_larger_capacity_test_" +\
        dataset_name_to_index(dataset_name) + ".png"
plt.savefig(filename_out)


In [ ]:

fig, ax = plt.subplots(1)
plot_idx = 0

# plot 3: loss mean with larger capacity
ax.plot(capacities, prediction_losses_mean_test)
ax.legend(fontsize=8)
ax.legend()
ax.legend(['loss mean with larger capacity'])
ax.set_xlabel(x_label_name_shannon_capacity)
ax.set_ylabel("Prediction loss mean (test data)")
ax.set_title(dataset_name_to_index(dataset_name))

filename_out = output_path + "/prediction_loss_mean_larger_capacity_test_" +\
        dataset_name_to_index(dataset_name) + ".png"
plt.savefig(filename_out)


In [ ]:

fig, ax = plt.subplots(1)
plot_idx = 0

# plot 4: loss mean with larger capacity (test data + DB query)
ax.plot(capacities, loss_variance_testset_on_threshold_capacity)
ax.legend(fontsize=8)
ax.legend()
ax.legend(['loss mean with larger capacity (test data + DB query)'])
ax.set_xlabel(x_label_name_shannon_capacity)
ax.set_ylabel("Prediction loss variance after applying (N/K)^2 (test data)")
ax.set_title(dataset_name_to_index(dataset_name))

filename_out = output_path + "/prediction_loss_mean_larger_capacity_test_data_with_db_query_" +\
        dataset_name_to_index(dataset_name) + ".png"
plt.savefig(filename_out)


In [ ]:


fig, ax = plt.subplots(1)
plot_idx = 0

# plot 5: Mean of number of queries 
ax.plot(capacities, mean_num_queries_in_subspace_on_threshold_capacity)
ax.legend(fontsize=8)
ax.legend()
ax.legend(['Mean num of queries in validation data in SNR'])
ax.set_xlabel(x_label_name_shannon_capacity)
ax.set_ylabel("Mean num queries in validation for SNR")
ax.set_title(dataset_name_to_index(dataset_name))

filename_out = output_path + "/prediction_mean_num_queries_in_subspace_validation_data_" +\
        dataset_name_to_index(dataset_name) + ".png"
plt.savefig(filename_out)


In [ ]:


fig, ax = plt.subplots(1)
plot_idx = 0

# plot 5: Mean of log2{max-min}
ax.plot(capacities, mean_num_bits_required_testset_on_threshold_capacity)
ax.legend(fontsize=8)
ax.legend()
ax.legend(['Mean log2{max-min} validation data (larger capacity)'])
ax.set_xlabel(x_label_name_shannon_capacity)
ax.set_ylabel("Mean of of log2{max-min} in validation data")
ax.set_title(dataset_name_to_index(dataset_name))

filename_out = output_path + "/prediction_mean_log2_max_minus_min_validation_data_" +\
        dataset_name_to_index(dataset_name) + ".png"
plt.savefig(filename_out)


In [ ]:

fig, ax = plt.subplots(1)
plot_idx = 0

plt.gcf().set_size_inches(10, 10, forward=True) 

# plot 6: CI (Confidence Interval)
confidence_interval_per_capacity_length_text = 'CI low bound vs. high bound length, division_factor=' +\
                                               str(stats_norm_interval_division_factor_const)
#print(confidence_interval_per_capacity_low_bound)
ax.plot(capacities, confidence_interval_per_capacity_low_bound)
ax.plot(capacities, confidence_interval_per_capacity_high_bound)
ax.plot(capacities, mean_target_actual_value_on_capacity)
ax.plot(capacities, confidence_interval_per_capacity_length)


ax.plot(capacities, median_target_actual_value_on_capacity)
ax.plot(capacities, percentile_0_25_target_actual_value_on_capacity)
ax.plot(capacities, percentile_0_50_target_actual_value_on_capacity)
ax.plot(capacities, percentile_0_75_target_actual_value_on_capacity)

ax.legend(fontsize=8)
ax.legend()
ax.legend(['CI in validation data with larger capacity LOW bound', 'CI in validation data with larger capacity HIGH bound',
                     'Mean of actual target value >= capacity', confidence_interval_per_capacity_length_text,
                     'Median of actual target value >= capacity',
                     '0.25 percentile of actual target value >= capacity',
                     '0.50 percentile of actual target value >= capacity',
                     '0.75 percentile of actual target value >= capacity'])

ax.set_xlabel(x_label_name_shannon_capacity)
ax.set_ylabel("CI in validation data")
ax.set_title(dataset_name_to_index(dataset_name))

filename_out = output_path + "/prediction_confidence_interval_validation_data_" +\
        dataset_name_to_index(dataset_name) + ".png"
plt.savefig(filename_out)


In [ ]:


# Calculate the interval length mean average for each shannon capacity and plot in plot 6 and put a number in each shannon capacity

fig, ax = plt.subplots(1)
plot_idx = 0

# plot 7: CI (Confidence Interval coverage ratio)
#print(confidence_interval_per_capacity_low_bound)
ax.plot(capacities, confidence_interval_per_capacity_coverage_ratio)
ax.legend(fontsize=8)
ax.legend()
ax.legend(['CI coverage ratio (by Shannon''s capacity)'])
ax.set_xlabel(x_label_name_shannon_capacity)
ax.set_ylabel("CI in validation data")
ax.set_title(dataset_name_to_index(dataset_name))

filename_out = output_path + "/prediction_confidence_interval_coverage_ratio_validation_data_" +\
        dataset_name_to_index(dataset_name) + ".png"
plt.savefig(filename_out)


In [ ]:
fig, ax = plt.subplots()
ax.scatter(capacities, loss_means)

plt.gca().update(dict(title='Loss Mean vs. Shannon Capacity', xlabel='x', ylabel='y'))
ax.legend(fontsize=8)
ax.set_xlabel(x_label_name_shannon_capacity)
ax.set_ylabel("Loss Mean")


In [ ]:
classification_df.to_csv(csv_vectors_output_filename, index = False)

classification_df.tail(10)

In [ ]:
def MPIW_calculate(low_bound_per_capacity, high_bound_per_capacity, queries_per_capacity):
    mpiw = 0
    total_queries = 0
    for idx in range(0, len(queries_per_capacity)):
        width = high_bound_per_capacity[idx] - low_bound_per_capacity[idx]
        mpiw += (width * queries_per_capacity[idx])
        total_queries += queries_per_capacity[idx]

    mpiw /= total_queries
    
    return mpiw

In [ ]:
# Calculate coverage for each PI [PICP]
HunchPI_coverage = total_hunch_pi_coverage_queries / total_queries

# Calculate the MPIW
HunchPI_MPIW = MPIW_calculate(df_snr_results['confidence_interval_per_capacity_low_bound'],
                              df_snr_results['confidence_interval_per_capacity_high_bound'],
                              df_snr_results['queries_per_capacity_total'])

print("Dataset name = ",  dataset_name)
print("Dataset index = ",  dataset_name_to_index(dataset_name))
print("total_hunch_pi_coverage_queries = ", total_hunch_pi_coverage_queries)
print("total_queries = ", total_queries)
print("HunchPI_coverage [PICP] = ", HunchPI_coverage)
print("HunchPI [log10(MPIW)] = ", np.log10(HunchPI_MPIW))
